# Pruebas

In [4]:
import numpy as np
from dataclasses import dataclass
from typing import Dict, List, Optional
import pandas as pd

@dataclass
class ComponenteMedicion:
    """Clase para almacenar mediciones de un componente"""
    nombre: str
    voltaje_ac: Optional[Dict[str, float]] = None  # {'L1-L2': 220, 'L2-L3': 220, ...}
    corriente_ac: Optional[Dict[str, float]] = None  # {'L1': 30, 'L2': 28, ...}
    voltaje_dc: Optional[float] = None
    corriente_dc: Optional[float] = None
    potencia_activa: Optional[float] = None  # kW
    potencia_reactiva: Optional[float] = None  # kVAR
    potencia_aparente: Optional[float] = None  # kVA
    factor_potencia: Optional[float] = None
    eficiencia: float = 1.0  # Eficiencia del componente (0-1)

class BalancePotenciaNodo:
    """
    Sistema para calcular y verificar el balance de potencia en un nodo
    desde el transformador hasta los PDB
    """

    # Eficiencias típicas de componentes
    EFICIENCIAS = {
        'transformador': 0.98,
        'ats': 0.99,
        'ml': 0.995,
        'rectificador': 0.92,  # AC->DC tiene pérdidas significativas
        'pdb': 0.98
    }

    def __init__(self):
        self.componentes: Dict[str, ComponenteMedicion] = {}
        self.flujo_potencia: Dict[str, float] = {}

    def agregar_componente(self, componente: ComponenteMedicion):
        """Registra un componente en el sistema"""
        self.componentes[componente.nombre] = componente

    def calcular_potencia_ac_trifasica(self, voltajes: Dict[str, float],
                                       corrientes: Dict[str, float],
                                       fp: float = 0.9) -> Dict[str, float]:
        """
        Calcula potencias en sistema trifásico

        Args:
            voltajes: {'L1-L2': V12, 'L2-L3': V23, 'L3-L1': V31}
            corrientes: {'L1': I1, 'L2': I2, 'L3': I3}
            fp: Factor de potencia (default 0.9)

        Returns:
            {'P_kW': potencia_activa, 'Q_kVAR': reactiva, 'S_kVA': aparente}
        """
        # Voltaje línea-línea promedio
        v_ll_promedio = np.mean(list(voltajes.values()))

        # Corriente promedio
        i_promedio = np.mean(list(corrientes.values()))

        # Potencia aparente trifásica: S = √3 × V_LL × I
        S_kVA = (np.sqrt(3) * v_ll_promedio * i_promedio) / 1000

        # Potencia activa: P = S × FP
        P_kW = S_kVA * fp

        # Potencia reactiva: Q = S × sin(arccos(FP))
        Q_kVAR = S_kVA * np.sin(np.arccos(fp))

        return {
            'P_kW': round(P_kW, 2),
            'Q_kVAR': round(Q_kVAR, 2),
            'S_kVA': round(S_kVA, 2),
            'V_LL_avg': round(v_ll_promedio, 2),
            'I_avg': round(i_promedio, 2)
        }

    def calcular_potencia_dc(self, voltaje_dc: float, corriente_dc: float) -> float:
        """
        Calcula potencia en DC

        Args:
            voltaje_dc: Voltaje DC en V
            corriente_dc: Corriente DC en A

        Returns:
            Potencia en kW
        """
        return (voltaje_dc * corriente_dc) / 1000

    def verificar_balance_transformador_ats(self) -> Dict:
        """
        Verifica el balance entre Transformador y ATS
        TR mide potencia directamente, ATS mide V e I
        """
        tr = self.componentes.get('TR')
        ats = self.componentes.get('ATS')

        if not tr or not ats:
            return {'error': 'Faltan mediciones de TR o ATS'}

        # Potencia del transformador (medida directamente)
        P_tr = tr.potencia_aparente  # kVA del sensor

        # Potencia calculada del ATS
        potencias_ats = self.calcular_potencia_ac_trifasica(
            ats.voltaje_ac,
            ats.corriente_ac,
            ats.factor_potencia or 0.9
        )

        # Potencia esperada en ATS considerando eficiencia
        P_ats_esperada = P_tr * self.EFICIENCIAS['ats']
        P_ats_medida = potencias_ats['S_kVA']

        # Diferencia porcentual
        diferencia_pct = abs(P_ats_medida - P_ats_esperada) / P_ats_esperada * 100

        return {
            'P_TR_kVA': P_tr,
            'P_ATS_medida_kVA': P_ats_medida,
            'P_ATS_esperada_kVA': round(P_ats_esperada, 2),
            'diferencia_pct': round(diferencia_pct, 2),
            'balance_ok': diferencia_pct < 5,  # Tolerancia 5%
            'perdidas_kW': round(P_tr - P_ats_medida, 2)
        }

    def verificar_balance_ats_ml(self) -> Dict:
        """
        Verifica balance entre ATS y ML (Main Line)
        Ambos en AC
        """
        ats = self.componentes.get('ATS')
        ml = self.componentes.get('ML')

        if not ats or not ml:
            return {'error': 'Faltan mediciones de ATS o ML'}

        # Calcular potencias
        pot_ats = self.calcular_potencia_ac_trifasica(
            ats.voltaje_ac, ats.corriente_ac, ats.factor_potencia or 0.9
        )

        pot_ml = self.calcular_potencia_ac_trifasica(
            ml.voltaje_ac, ml.corriente_ac, ml.factor_potencia or 0.9
        )

        # Potencia esperada en ML
        P_ml_esperada = pot_ats['S_kVA'] * self.EFICIENCIAS['ml']
        diferencia_pct = abs(pot_ml['S_kVA'] - P_ml_esperada) / P_ml_esperada * 100

        return {
            'P_ATS_kVA': pot_ats['S_kVA'],
            'P_ML_medida_kVA': pot_ml['S_kVA'],
            'P_ML_esperada_kVA': round(P_ml_esperada, 2),
            'diferencia_pct': round(diferencia_pct, 2),
            'balance_ok': diferencia_pct < 5,
            'detalle_ml': pot_ml
        }

    def verificar_distribucion_rectificadores(self) -> Dict:
        """
        Verifica cómo el ML distribuye potencia entre rectificadores
        ML (AC) -> Rectificadores (DC)
        """
        ml = self.componentes.get('ML')
        rect1 = self.componentes.get('RECT1')
        rect2 = self.componentes.get('RECT2')

        if not ml:
            return {'error': 'Falta medición de ML'}

        # Potencia total disponible del ML
        pot_ml = self.calcular_potencia_ac_trifasica(
            ml.voltaje_ac, ml.corriente_ac, ml.factor_potencia or 0.9
        )
        P_ml_disponible = pot_ml['P_kW']  # Usamos potencia activa

        resultados = {
            'P_ML_disponible_kW': P_ml_disponible,
            'rectificadores': {}
        }

        # Calcular consumo de cada rectificador (en DC)
        potencia_total_rect = 0

        for nombre, rect in [('RECT1', rect1), ('RECT2', rect2)]:
            if rect and rect.voltaje_dc and rect.corriente_dc:
                P_dc = self.calcular_potencia_dc(rect.voltaje_dc, rect.corriente_dc)

                # Potencia AC equivalente (considerando eficiencia del rectificador)
                P_ac_equiv = P_dc / self.EFICIENCIAS['rectificador']

                potencia_total_rect += P_ac_equiv

                resultados['rectificadores'][nombre] = {
                    'P_DC_kW': round(P_dc, 2),
                    'P_AC_equiv_kW': round(P_ac_equiv, 2),
                    'V_DC': rect.voltaje_dc,
                    'I_DC': rect.corriente_dc,
                    'porcentaje_ML': round((P_ac_equiv / P_ml_disponible) * 100, 1)
                }

        # Potencia restante disponible
        P_restante = P_ml_disponible - potencia_total_rect
        pct_utilizado = (potencia_total_rect / P_ml_disponible) * 100

        resultados['resumen'] = {
            'P_total_rectificadores_kW': round(potencia_total_rect, 2),
            'P_restante_kW': round(P_restante, 2),
            'porcentaje_utilizado': round(pct_utilizado, 1),
            'capacidad_disponible': pct_utilizado < 80  # Alerta si >80%
        }

        return resultados

    def verificar_distribucion_pdbs(self) -> Dict:
        """
        Verifica cómo los rectificadores distribuyen a los PDBs
        Rectificadores (DC) -> PDBs (DC) -> Racks
        """
        rect1 = self.componentes.get('RECT1')
        rect2 = self.componentes.get('RECT2')
        pdb1 = self.componentes.get('PDB1')
        pdb2 = self.componentes.get('PDB2')

        resultados = {'rectificadores': {}, 'pdbs': {}}

        # Potencia disponible por rectificador
        for nombre, rect in [('RECT1', rect1), ('RECT2', rect2)]:
            if rect and rect.voltaje_dc and rect.corriente_dc:
                P_rect = self.calcular_potencia_dc(rect.voltaje_dc, rect.corriente_dc)
                resultados['rectificadores'][nombre] = {
                    'P_disponible_kW': round(P_rect, 2)
                }

        # Potencia consumida por PDBs (si se tienen mediciones)
        # Nota: Según tu diagrama, PDB1 se conecta a RECT1 y PDB2 a RECT2
        conexiones = [('PDB1', pdb1, 'RECT1'), ('PDB2', pdb2, 'RECT2')]

        for nombre_pdb, pdb, rect_fuente in conexiones:
            if pdb and pdb.corriente_dc:
                # Asumimos voltaje nominal de 48V si no está disponible
                V_dc = pdb.voltaje_dc or 48.0
                P_pdb = self.calcular_potencia_dc(V_dc, pdb.corriente_dc)

                # Potencia del rectificador fuente
                rect = self.componentes.get(rect_fuente)
                P_rect = 0
                if rect and rect.voltaje_dc and rect.corriente_dc:
                    P_rect = self.calcular_potencia_dc(rect.voltaje_dc, rect.corriente_dc)

                pct_consumo = (P_pdb / P_rect * 100) if P_rect > 0 else 0

                resultados['pdbs'][nombre_pdb] = {
                    'P_consumida_kW': round(P_pdb, 2),
                    'V_DC': V_dc,
                    'I_DC': pdb.corriente_dc,
                    'rectificador_fuente': rect_fuente,
                    'porcentaje_rectificador': round(pct_consumo, 1),
                    'racks_alimentados': 'Fila 1' if nombre_pdb == 'PDB1' else 'Fila 2'
                }

        return resultados

    def generar_reporte_completo(self) -> pd.DataFrame:
        """
        Genera un reporte completo del balance de potencia
        """
        reportes = []

        # Verificación TR -> ATS
        bal_tr_ats = self.verificar_balance_transformador_ats()
        if 'error' not in bal_tr_ats:
            reportes.append({
                'Tramo': 'TR → ATS',
                'Potencia Entrada (kVA)': bal_tr_ats['P_TR_kVA'],
                'Potencia Salida (kVA)': bal_tr_ats['P_ATS_medida_kVA'],
                'Esperada (kVA)': bal_tr_ats['P_ATS_esperada_kVA'],
                'Diferencia (%)': bal_tr_ats['diferencia_pct'],
                'Estado': '✓ OK' if bal_tr_ats['balance_ok'] else '✗ Revisar'
            })

        # Verificación ATS -> ML
        bal_ats_ml = self.verificar_balance_ats_ml()
        if 'error' not in bal_ats_ml:
            reportes.append({
                'Tramo': 'ATS → ML',
                'Potencia Entrada (kVA)': bal_ats_ml['P_ATS_kVA'],
                'Potencia Salida (kVA)': bal_ats_ml['P_ML_medida_kVA'],
                'Esperada (kVA)': bal_ats_ml['P_ML_esperada_kVA'],
                'Diferencia (%)': bal_ats_ml['diferencia_pct'],
                'Estado': '✓ OK' if bal_ats_ml['balance_ok'] else '✗ Revisar'
            })

        return pd.DataFrame(reportes)

In [5]:
def ejemplo_nodo_ideo_cali():

    sistema = BalancePotenciaNodo()


    tr = ComponenteMedicion(
        nombre='TR',
        potencia_aparente=21.00,
        potencia_activa=21.0,
        factor_potencia=0.98
    )


    ats = ComponenteMedicion(
        nombre='ATS',
        voltaje_ac={'R-S': 214, 'S-T': 214, 'T-R': 215},
        corriente_ac={'R': 58, 'S': 58, 'T': 59},
        factor_potencia=0.98
    )

    # Datos del ML (podría ser igual o derivado del ATS)
    ml = ComponenteMedicion(
        nombre='ML',
        voltaje_ac={'R-S': 214.93, 'S-T': 214.35, 'T-R': 219.85},
        corriente_ac={'R': 58.46, 'S': 59.46, 'T': 66.25},
        factor_potencia=0.9
    )

    # Rectificador 1 (ELTEK 1 - 21 sensores)
    rect1 = ComponenteMedicion(
        nombre='RECT1',
        voltaje_dc=54.47,  # V típico de sistema 48V
        corriente_dc=119.0  # A
    )

    # Rectificador 2 (ELTEK 2)
    rect2 = ComponenteMedicion(
        nombre='RECT2',
        voltaje_dc=54.47,
        corriente_dc=157.0
    )

    # PDB1 (alimenta Racks Fila 1)
    pdb1 = ComponenteMedicion(
        nombre='PDB1',
        voltaje_dc=48.0,
        corriente_dc=75.0
    )

    # PDB2 (alimenta Racks Fila 2)
    pdb2 = ComponenteMedicion(
        nombre='PDB2',
        voltaje_dc=48.0,
        corriente_dc=68.0
    )

    # Agregar componentes al sistema
    for comp in [tr, ats, ml, rect1, rect2, pdb1, pdb2]:
        sistema.agregar_componente(comp)

    # Generar verificaciones
    print("=" * 60)
    print("REPORTE DE BALANCE DE POTENCIA - NODO IDEO CALI")
    print("=" * 60)

    print("\n1. VERIFICACIÓN TR → ATS:")
    print("-" * 60)
    bal_tr_ats = sistema.verificar_balance_transformador_ats()
    for k, v in bal_tr_ats.items():
        print(f"  {k}: {v}")

    print("\n2. VERIFICACIÓN ATS → ML:")
    print("-" * 60)
    bal_ats_ml = sistema.verificar_balance_ats_ml()
    for k, v in bal_ats_ml.items():
        print(f"  {k}: {v}")

    print("\n3. DISTRIBUCIÓN ML → RECTIFICADORES:")
    print("-" * 60)
    dist_rect = sistema.verificar_distribucion_rectificadores()
    for k, v in dist_rect.items():
        if isinstance(v, dict):
            print(f"\n  {k}:")
            for k2, v2 in v.items():
                print(f"    {k2}: {v2}")
        else:
            print(f"  {k}: {v}")

    print("\n4. DISTRIBUCIÓN RECTIFICADORES → PDBs:")
    print("-" * 60)
    dist_pdb = sistema.verificar_distribucion_pdbs()
    for k, v in dist_pdb.items():
        print(f"\n  {k}:")
        for k2, v2 in v.items():
            print(f"    {k2}: {v2}")

    print("\n" + "=" * 60)
    print("TABLA RESUMEN DE BALANCE:")
    print("=" * 60)
    df_reporte = sistema.generar_reporte_completo()
    print(df_reporte.to_string(index=False))


if __name__ == "__main__":
    ejemplo_nodo_ideo_cali()

REPORTE DE BALANCE DE POTENCIA - NODO IDEO CALI

1. VERIFICACIÓN TR → ATS:
------------------------------------------------------------
  P_TR_kVA: 21.0
  P_ATS_medida_kVA: 21.66
  P_ATS_esperada_kVA: 20.79
  diferencia_pct: 4.18
  balance_ok: True
  perdidas_kW: -0.66

2. VERIFICACIÓN ATS → ML:
------------------------------------------------------------
  P_ATS_kVA: 21.66
  P_ML_medida_kVA: 23.01
  P_ML_esperada_kVA: 21.55
  diferencia_pct: 6.77
  balance_ok: False
  detalle_ml: {'P_kW': np.float64(20.71), 'Q_kVAR': np.float64(10.03), 'S_kVA': np.float64(23.01), 'V_LL_avg': np.float64(216.38), 'I_avg': np.float64(61.39)}

3. DISTRIBUCIÓN ML → RECTIFICADORES:
------------------------------------------------------------
  P_ML_disponible_kW: 20.71

  rectificadores:
    RECT1: {'P_DC_kW': 6.48, 'P_AC_equiv_kW': 7.05, 'V_DC': 54.47, 'I_DC': 119.0, 'porcentaje_ML': np.float64(34.0)}
    RECT2: {'P_DC_kW': 8.55, 'P_AC_equiv_kW': 9.3, 'V_DC': 54.47, 'I_DC': 157.0, 'porcentaje_ML': np.float

In [6]:
import math

class EvaluadorTecnico:
    def __init__(self, config_nodo):
        self.config = config_nodo

    def calcular_impacto_nuevo_equipo(self, datos_tiempo_real, nuevo_equipo):
        """
        datos_tiempo_real: Diccionario con los últimos valores de los sensores (TR, ML, Rect).
        nuevo_equipo: Diccionario con specs del equipo (Watts, Voltaje).
        """
        resultados = {
            "aprobado": False,
            "alertas": [],
            "detalles": {}
        }

        # --- PASO 0: Entender el equipo nuevo ---
        potencia_equipo_w = nuevo_equipo['potencia_watts']
        voltaje_dc_nom = 54.0  # Voltaje típico de flotación

        # Corriente que consumirá el equipo en el lado DC
        corriente_nueva_dc = potencia_equipo_w / voltaje_dc_nom

        # --- PASO 1: Verificación en Rectificadores (Lado DC) ---
        # Capacidad total instalada (Slide 16: 21 módulos de supongamos 2000W o 3000W)
        # Asumamos configuración N+1 (dejamos 1 módulo de backup)
        capacidad_rect_total_amps = self.config['capacidad_rectificadores_amps']
        capacidad_rect_n1_amps = capacidad_rect_total_amps - self.config['capacidad_modulo_individual_amps']

        corriente_actual_dc = datos_tiempo_real['rectificador_total_amps']

        futura_corriente_dc = corriente_actual_dc + corriente_nueva_dc

        if futura_corriente_dc > capacidad_rect_n1_amps:
            resultados["alertas"].append(f"FALLO DC: La corriente proyectada ({futura_corriente_dc:.2f}A) excede la capacidad N+1 ({capacidad_rect_n1_amps}A).")
            check_dc = False
        else:
            check_dc = True

        # --- PASO 2: Verificación Impacto Aguas Arriba (AC) ---
        # Convertimos la carga DC a AC considerando eficiencia del rectificador
        eficiencia_rect = 0.94 # Típico Flatpack2
        potencia_ac_adicional_w = potencia_equipo_w / eficiencia_rect

        # Obtenemos la carga actual del Transformador (Slide 30 - Sensor 16 Potencia Aparente)
        kva_actual_tr = datos_tiempo_real['tr_kva_total']

        # Convertimos la nueva carga AC (Watts) a kVA
        # Asumimos un Factor de Potencia para el equipo nuevo (típicamente 0.98 en fuentes modernas)
        fp_nuevo_equipo = 0.98
        kva_adicional = (potencia_ac_adicional_w / 1000) / fp_nuevo_equipo

        kva_futuro_total = kva_actual_tr + kva_adicional

        # Límite del TR (Slide 8: 75 kVA)
        limite_tr_seguridad = self.config['capacidad_tr_kva'] * 0.90 # Margen de seguridad del 90%

        if kva_futuro_total > limite_tr_seguridad:
            resultados["alertas"].append(f"FALLO AC: El transformador quedaría al { (kva_futuro_total/self.config['capacidad_tr_kva'])*100:.1f}% de capacidad.")
            check_ac = False
        else:
            check_ac = True

        # --- PASO 3: Verificación de Breakers (ML) ---
        # Slide 15: Breaker de Rectificadores es de 150A por circuito (hay 2 breakers)
        # Estimamos corriente AC adicional por fase
        voltaje_fase_fase = 220
        # Formula trifásica: I = P / (sqrt(3) * V * FP)
        corriente_ac_adicional_fase = potencia_ac_adicional_w / (math.sqrt(3) * voltaje_fase_fase * fp_nuevo_equipo)

        corriente_actual_ml = datos_tiempo_real['ml_corriente_promedio'] # Sensor ML CURRENT AC
        corriente_futura_ml = corriente_actual_ml + corriente_ac_adicional_fase

        if corriente_futura_ml > self.config['breaker_ml_rectificadores_amps']:
             resultados["alertas"].append(f"FALLO BREAKER: Se excedería la capacidad del breaker del ML.")
             check_breaker = False
        else:
             check_breaker = True

        # --- CONCLUSIÓN ---
        resultados["aprobado"] = check_dc and check_ac and check_breaker
        resultados["detalles"] = {
            "aumento_corriente_dc": round(corriente_nueva_dc, 2),
            "aumento_potencia_ac_kw": round(potencia_ac_adicional_w/1000, 2),
            "kva_proyectado_tr": round(kva_futuro_total, 2),
            "capacidad_restante_tr_kva": round(self.config['capacidad_tr_kva'] - kva_futuro_total, 2)
        }

        return resultados

# ==========================================
# EJEMPLO DE USO CON TUS DATOS DEL PDF
# ==========================================

# 1. Configuración Estática del Nodo (Basado en PDF IDEO Cali)
config_ideo = {
    "capacidad_tr_kva": 75.0,            # Slide 8
    "capacidad_rectificadores_amps": 1000, # Suponiendo 20 modulos x 50A
    "capacidad_modulo_individual_amps": 50,
    "breaker_ml_rectificadores_amps": 150 # Slide 15
}

# 2. Datos simulados obtenidos de tu CSV/API (Estado Actual)
datos_sensores = {
    "tr_kva_total": 21.0,           # Dato que diste al principio
    "rectificador_total_amps": 180.0, # Suma de carga actual
    "ml_corriente_promedio": 60.0     # Amperios en AC
}

# 3. Equipo Nuevo a Instalar (Ejemplo: Un Switch Core Grande)
nuevo_equipo_solicitud = {
    "nombre": "Cisco Nexus Core",
    "potencia_watts": 2500 # consume 2.5 kW
}

# 4. Ejecutar evaluación
sistema = EvaluadorTecnico(config_ideo)
informe = sistema.calcular_impacto_nuevo_equipo(datos_sensores, nuevo_equipo_solicitud)

print(f"Estado de Aprobación: {'APROBADO' if informe['aprobado'] else 'RECHAZADO'}")
print("Detalles Técnicos:", informe['detalles'])
for alerta in informe['alertas']:
    print("ALERTA:", alerta)

Estado de Aprobación: APROBADO
Detalles Técnicos: {'aumento_corriente_dc': 46.3, 'aumento_potencia_ac_kw': 2.66, 'kva_proyectado_tr': 23.71, 'capacidad_restante_tr_kva': 51.29}


In [7]:
import math

class EvaluadorNodoIdeo:
    def __init__(self):
        #CONFIGURACIÓN FÍSICA
        self.VOLTAJE_DC = 54.0
        self.VOLTAJE_AC_FASE = 220.0
        self.FP_EQUIPOS_NUEVOS = 0.98
        self.EFICIENCIA_RECT = 0.94

        # Valores límites
        self.LIMITES = {
            "tr_kva_max": 75.0,
            "breaker_ml_rectificador_amps": 125.0,  # Breaker que alimenta al rectificador en el ML
            "capacidad_modulo_rectificador_w": 3000.0,
            "totalizador_pdb_amps": 150.0  # Límite de entrada del PDB
        }

    def procesar_datos_api(self, json_tr, json_ml, json_rect1, json_rect2):

        datos = {}

        # TR
        sensor_kva = next(s for s in json_tr if "POTENCIA APARENTE" in s['label'])
        datos['tr_kva_actual'] = float(sensor_kva['value'].replace(" kVA", ""))

        # ML: Promediamos las 3 fases
        corrientes_ml = [
            float(next(s for s in json_ml if "CURRENT AC R" in s['label'])['value'].split()[0]),
            float(next(s for s in json_ml if "CURRENT AC S" in s['label'])['value'].split()[0]),
            float(next(s for s in json_ml if "CURRENT AC T" in s['label'])['value'].split()[0])
        ]
        datos['ml_amps_ac_avg'] = sum(corrientes_ml) / 3


        # Rectificador 1
        datos['r1_amps_dc'] = float(next(s for s in json_rect1 if "CORRIENTE DC DEL SISTEMA" in s['label'])['value'].split()[0])
        datos['r1_modulos'] = float(next(s for s in json_rect1 if "RECTIFICADORES INSTALADOS" in s['label'])['value'])

        # Rectificador 2
        datos['r2_amps_dc'] = float(next(s for s in json_rect2 if "CORRIENTE DC DEL SISTEMA" in s['label'])['value'].split()[0])
        datos['r2_modulos'] = float(next(s for s in json_rect2 if "RECTIFICADORES INSTALADOS" in s['label'])['value'])

        return datos

    def evaluar_instalacion(self, datos_tiempo_real, nuevo_equipo_watts):
        """
        Realiza el balance de potencia y comprobaciones de seguridad
        """
        informe = {"aprobado": False, "checks": [], "mensaje_final": ""}

        # --- CÁLCULOS DEL NUEVO EQUIPO ---
        corriente_nueva_dc = nuevo_equipo_watts / self.VOLTAJE_DC

        # Potencia AC que consumirá el equipo (considerando pérdidas del rectificador)
        potencia_ac_req_w = nuevo_equipo_watts / self.EFICIENCIA_RECT
        # Corriente AC por fase (trifásica)
        corriente_ac_req_fase = potencia_ac_req_w / (math.sqrt(3) * self.VOLTAJE_AC_FASE * 0.95)
        # Impacto en kVA
        kva_req = (potencia_ac_req_w / 1000) / self.FP_EQUIPOS_NUEVOS

        # ---------------------------------------------------------
        # CHECK 1: REDUNDANCIA RECTIFICADORES (CRÍTICO)
        # ---------------------------------------------------------
        # Escenario: Falla Rectificador 1. ¿El Rectificador 2 puede con TODO?
        # Capacidad R2 (N+1) = (Módulos - 1) * Amperaje por módulo
        # 3000W / 54V = 55.5 Amperios por módulo aprox.
        amps_por_modulo = self.LIMITES['capacidad_modulo_rectificador_w'] / self.VOLTAJE_DC

        capacidad_total_r2 = (datos_tiempo_real['r2_modulos'] - 1) * amps_por_modulo # Dejamos 1 de reserva

        carga_total_sitio = datos_tiempo_real['r1_amps_dc'] + datos_tiempo_real['r2_amps_dc']
        carga_futura_total = carga_total_sitio + corriente_nueva_dc

        if carga_futura_total < capacidad_total_r2:
            informe['checks'].append(f"✅ REDUNDANCIA DC: OK. Carga total ({carga_futura_total:.1f}A) soportada por R2 ({capacidad_total_r2:.1f}A).")
        else:
            informe['checks'].append(f"❌ REDUNDANCIA DC: FALLO. Si R1 falla, R2 colapsaría. Capacidad R2: {capacidad_total_r2:.1f}A, Carga necesaria: {carga_futura_total:.1f}A")
            return informe # Abortar evaluación

        # ---------------------------------------------------------
        # CHECK 2: CAPACIDAD DEL TRANSFORMADOR (AC)
        # ---------------------------------------------------------
        kva_futuro = datos_tiempo_real['tr_kva_actual'] + kva_req
        uso_tr_pct = (kva_futuro / self.LIMITES['tr_kva_max']) * 100

        if uso_tr_pct < 90: # Margen seguridad
            informe['checks'].append(f"✅ TRANSFORMADOR: OK. Uso proyectado: {uso_tr_pct:.1f}% ({kva_futuro:.1f} kVA).")
        else:
            informe['checks'].append(f"❌ TRANSFORMADOR: SOBRECARGA. Uso proyectado: {uso_tr_pct:.1f}%.")
            return informe

        # ---------------------------------------------------------
        # CHECK 3: BREAKER AC DEL RECTIFICADOR (ML)
        # ---------------------------------------------------------
        # Asumimos que la nueva carga se va mitad a R1 y mitad a R2 en operación normal
        corriente_ac_actual_est = (datos_tiempo_real['r2_amps_dc'] * self.VOLTAJE_DC) / (math.sqrt(3) * self.VOLTAJE_AC_FASE * 0.95 * self.EFICIENCIA_RECT)
        corriente_ac_futura_breaker = corriente_ac_actual_est + (corriente_ac_req_fase / 2)

        if corriente_ac_futura_breaker < self.LIMITES['breaker_ml_rectificador_amps']:
            informe['checks'].append(f"✅ BREAKER ML: OK. Corriente estimada: {corriente_ac_futura_breaker:.1f}A (Límite {self.LIMITES['breaker_ml_rectificador_amps']}A).")
        else:
            informe['checks'].append(f"❌ BREAKER ML: PELIGRO. Se excede capacidad del breaker de 125A.")
            return informe

        informe['aprobado'] = True
        informe['mensaje_final'] = "Instalación VIABLE técnicamente."
        return informe

# ==========================================
# SIMULACIÓN CON TUS DATOS REALES (EXTRAÍDOS DE LOS JSON)
# ==========================================

# Datos "mockeados" basados en lo que leí de tus capturas
datos_api_simulados = {
    'tr_kva_actual': 21.0,      # Captura pág 10
    'ml_amps_ac_avg': 61.0,     # Promedio de pág 13
    'r1_amps_dc': 119.0,        # Captura pág 19
    'r1_modulos': 21.0,         # Captura pág 16
    'r2_amps_dc': 157.0,        # Captura pág 25
    'r2_modulos': 19.0          # Captura pág 24 (OJO: Aquí usé el dato real de la API, no del PDF)
}

evaluador = EvaluadorNodoIdeo()

# Probamos instalando un equipo que consuma 2000 Watts
nuevo_equipo = 200

resultado = evaluador.evaluar_instalacion(datos_api_simulados, nuevo_equipo)

print(f"--- RESULTADO DE EVALUACIÓN PARA EQUIPO DE {nuevo_equipo}W ---")
for check in resultado['checks']:
    print(check)
print(f"\nCONCLUSIÓN: {resultado['mensaje_final']}")

--- RESULTADO DE EVALUACIÓN PARA EQUIPO DE 200W ---
✅ REDUNDANCIA DC: OK. Carga total (279.7A) soportada por R2 (1000.0A).
✅ TRANSFORMADOR: OK. Uso proyectado: 28.3% (21.2 kVA).
✅ BREAKER ML: OK. Corriente estimada: 25.2A (Límite 125.0A).

CONCLUSIÓN: Instalación VIABLE técnicamente.


# Evaluacion

In [8]:
import math
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from enum import Enum
import json

class NivelSeveridad(Enum):
    """Niveles de severidad para los checks"""
    CRITICO = "CRÍTICO"
    ADVERTENCIA = "ADVERTENCIA"
    INFO = "INFO"

@dataclass
class ResultadoCheck:
    """Resultado individual de una verificación"""
    nombre: str
    aprobado: bool
    severidad: NivelSeveridad
    mensaje: str
    valor_actual: float
    valor_limite: float
    porcentaje_uso: float
    recomendacion: Optional[str] = None

@dataclass
class DatosNodoTiempoReal:
    """Estructura de datos del nodo en tiempo real"""
    # Transformador
    tr_kva_actual: float

    # Main Line (ML)
    ml_amps_ac_r: float
    ml_amps_ac_s: float
    ml_amps_ac_t: float
    ml_voltaje_ac_rs: float
    ml_voltaje_ac_st: float
    ml_voltaje_ac_tr: float
    ml_fp: float

    # Rectificador 1
    r1_amps_dc: float
    r1_voltaje_dc: float
    r1_modulos_instalados: int
    r1_modulos_fallados: int
    r1_porcentaje_carga: float

    # Rectificador 2
    r2_amps_dc: float
    r2_voltaje_dc: float
    r2_modulos_instalados: int
    r2_modulos_fallados: int
    r2_porcentaje_carga: float

    # PDBs (opcional, si tienes datos)
    pdb1_amps_dc: Optional[float] = None
    pdb2_amps_dc: Optional[float] = None

    @property
    def ml_amps_ac_avg(self) -> float:
        """Corriente promedio del ML"""
        return (self.ml_amps_ac_r + self.ml_amps_ac_s + self.ml_amps_ac_t) / 3

    @property
    def ml_voltaje_ac_avg(self) -> float:
        """Voltaje línea-línea promedio"""
        return (self.ml_voltaje_ac_rs + self.ml_voltaje_ac_st + self.ml_voltaje_ac_tr) / 3

    @property
    def r1_modulos_operativos(self) -> int:
        """Módulos operativos del rectificador 1"""
        return self.r1_modulos_instalados - self.r1_modulos_fallados

    @property
    def r2_modulos_operativos(self) -> int:
        """Módulos operativos del rectificador 2"""
        return self.r2_modulos_instalados - self.r2_modulos_fallados

@dataclass
class EquipoNuevo:
    """Especificaciones del equipo a instalar"""
    nombre: str
    potencia_watts: float
    voltaje_operacion: float = 48.0  # Voltaje nominal DC
    redundancia_requerida: bool = True
    pdb_destino: Optional[str] = None  # "PDB1" o "PDB2"
    rack_destino: Optional[str] = None

class EvaluadorNodoIdeo:
    """
    Sistema de evaluación técnica para instalación de equipos
    en nodos de telecomunicaciones
    """

    def __init__(self):
        # CONFIGURACIÓN FÍSICA DEL NODO
        self.config = {
            # Sistema eléctrico
            "voltaje_dc_nominal": 48.0,
            "voltaje_dc_min": 43.0,
            "voltaje_dc_max": 57.0,
            "voltaje_ac_nominal": 220.0,
            "frecuencia_hz": 60.0,

            # Factores de conversión y eficiencia
            "fp_equipos_nuevos": 0.98,
            "eficiencia_rectificador": 0.94,
            "eficiencia_pdb": 0.98,

            # Márgenes de seguridad (%)
            "margen_seguridad_tr": 10,  # 90% uso máximo
            "margen_seguridad_rect": 20,  # 80% uso máximo
            "margen_seguridad_breaker": 20,  # 80% uso máximo
        }

        # LÍMITES DEL NODO IDEO CALI
        self.limites = {
            # Transformador
            "tr_kva_max": 75.0,

            # Main Line
            "breaker_ml_principal_amps": 200.0,  # Breaker principal del ML
            "breaker_ml_por_rectificador_amps": 125.0,  # Breaker que alimenta cada rectificador

            # Rectificadores (ELTEK SMARTPACK 2 TOUCH)
            "capacidad_modulo_rectificador_w": 3000.0,  # Flatpack2 48V/3kW
            "corriente_modulo_rectificador_a": 55.5,  # 3000W / 54V
            "voltaje_salida_rect_nominal": 54.0,

            # PDBs
            "totalizador_pdb_amps": 150.0,  # Límite de entrada del PDB
            "fusible_por_salida_pdb_amps": 20.0,  # Fusibles típicos por salida

            # Térmico (para futura implementación)
            "temperatura_sala_max": 27.0,  # °C
            "btu_por_rack": 5000.0,  # BTU/h
        }

    def parsear_datos_api(self, json_tr: List[Dict], json_ml: List[Dict],
                          json_rect1: List[Dict], json_rect2: List[Dict]) -> DatosNodoTiempoReal:
        """
        Parsea los JSON de la API de Data Center Expert
        y retorna una estructura de datos tipada
        """
        try:
            return DatosNodoTiempoReal(
                # TR
                tr_kva_actual=self._extraer_valor(json_tr, "POTENCIA APARENTE", "kVA"),

                # ML - Corrientes
                ml_amps_ac_r=self._extraer_valor(json_ml, "CURRENT AC R", "A"),
                ml_amps_ac_s=self._extraer_valor(json_ml, "CURRENT AC S", "A"),
                ml_amps_ac_t=self._extraer_valor(json_ml, "CURRENT AC T", "A"),

                # ML - Voltajes
                ml_voltaje_ac_rs=self._extraer_valor(json_ml, "VOLTAGE AC R-S", "V"),
                ml_voltaje_ac_st=self._extraer_valor(json_ml, "VOLTAGE AC S-T", "V"),
                ml_voltaje_ac_tr=self._extraer_valor(json_ml, "VOLTAGE AC T-R", "V"),
                ml_fp=0.98,  # Si no está disponible en API

                # Rectificador 1
                r1_amps_dc=self._extraer_valor(json_rect1, "CORRIENTE DC DEL SISTEMA", "A"),
                r1_voltaje_dc=self._extraer_valor(json_rect1, "VOLTAJE DC DEL SISTEMA", "V"),
                r1_modulos_instalados=int(self._extraer_valor(json_rect1, "RECTIFICADORES INSTALADOS", "")),
                r1_modulos_fallados=int(self._extraer_valor(json_rect1, "RECTIFICADORES FALLADOS", "")),
                r1_porcentaje_carga=self._extraer_valor(json_rect1, "PORCENTAJE DE CARGA", "%"),

                # Rectificador 2
                r2_amps_dc=self._extraer_valor(json_rect2, "CORRIENTE DC DEL SISTEMA", "A"),
                r2_voltaje_dc=self._extraer_valor(json_rect2, "VOLTAJE DC DEL SISTEMA", "V"),
                r2_modulos_instalados=int(self._extraer_valor(json_rect2, "RECTIFICADORES INSTALADOS", "")),
                r2_modulos_fallados=int(self._extraer_valor(json_rect2, "RECTIFICADORES FALLADOS", "")),
                r2_porcentaje_carga=self._extraer_valor(json_rect2, "PORCENTAJE DE CARGA", "%"),
            )
        except Exception as e:
            raise ValueError(f"Error parseando datos de API: {str(e)}")

    def _extraer_valor(self, json_lista: List[Dict], label_buscar: str, unidad: str) -> float:
        """Extrae un valor numérico de la lista de sensores"""
        sensor = next((s for s in json_lista if label_buscar in s['label']), None)
        if not sensor:
            raise ValueError(f"Sensor no encontrado: {label_buscar}")

        valor_str = sensor['value'].replace(f" {unidad}", "").strip()
        return float(valor_str)

    def calcular_impacto_equipo(self, equipo: EquipoNuevo,
                                datos: DatosNodoTiempoReal) -> Dict[str, float]:
        """
        Calcula el impacto del nuevo equipo en todos los componentes
        """
        impacto = {}

        # 1. IMPACTO EN DC
        impacto['corriente_dc_a'] = equipo.potencia_watts / equipo.voltaje_operacion
        impacto['potencia_dc_kw'] = equipo.potencia_watts / 1000

        # 2. IMPACTO EN AC (considerando pérdidas del rectificador)
        potencia_ac_req_w = equipo.potencia_watts / self.config['eficiencia_rectificador']
        impacto['potencia_ac_w'] = potencia_ac_req_w
        impacto['potencia_ac_kw'] = potencia_ac_req_w / 1000

        # Corriente AC por fase (sistema trifásico)
        V_ac = datos.ml_voltaje_ac_avg
        impacto['corriente_ac_fase_a'] = potencia_ac_req_w / (
            math.sqrt(3) * V_ac * self.config['fp_equipos_nuevos']
        )

        # 3. IMPACTO EN KVA (transformador)
        impacto['kva_adicional'] = (potencia_ac_req_w / 1000) / self.config['fp_equipos_nuevos']

        return impacto

    def check_redundancia_n_mas_1(self, equipo: EquipoNuevo,
                                   datos: DatosNodoTiempoReal,
                                   impacto: Dict) -> ResultadoCheck:
        """
        CHECK CRÍTICO: Verifica redundancia N+1 de rectificadores

        Escenario: Si falla el rectificador con MÁS carga, ¿el otro puede soportar TODO?
        """
        # Capacidad de cada rectificador (N+1 significa: dejar 1 módulo de reserva)
        amps_por_modulo = self.limites['corriente_modulo_rectificador_a']

        # Capacidad N+1 de cada rectificador
        capacidad_r1_n_mas_1 = max(0, datos.r1_modulos_operativos - 1) * amps_por_modulo
        capacidad_r2_n_mas_1 = max(0, datos.r2_modulos_operativos - 1) * amps_por_modulo

        # Carga total actual del sitio
        carga_actual_total = datos.r1_amps_dc + datos.r2_amps_dc

        # Carga futura total
        carga_futura_total = carga_actual_total + impacto['corriente_dc_a']

        # El rectificador con MENOR capacidad es el cuello de botella
        capacidad_minima = min(capacidad_r1_n_mas_1, capacidad_r2_n_mas_1)

        # Verificar si puede soportar toda la carga
        aprobado = carga_futura_total <= capacidad_minima
        porcentaje_uso = (carga_futura_total / capacidad_minima * 100) if capacidad_minima > 0 else 100

        mensaje = (
            f"Carga futura total: {carga_futura_total:.1f}A. "
            f"Capacidad mínima N+1: {capacidad_minima:.1f}A (R1: {capacidad_r1_n_mas_1:.1f}A, R2: {capacidad_r2_n_mas_1:.1f}A). "
        )

        if aprobado:
            mensaje += "Sistema puede operar con un rectificador fuera de servicio."
        else:
            mensaje += "¡PELIGRO! Si falla un rectificador, el otro NO puede soportar la carga."

        recomendacion = None
        if not aprobado:
            modulos_faltantes = math.ceil((carga_futura_total - capacidad_minima) / amps_por_modulo)
            recomendacion = f"Instalar al menos {modulos_faltantes} módulos adicionales en el rectificador con menor capacidad."
        elif porcentaje_uso > 80:
            recomendacion = "Considerar agregar módulos antes de futuras expansiones."

        return ResultadoCheck(
            nombre="Redundancia N+1 (Rectificadores)",
            aprobado=aprobado,
            severidad=NivelSeveridad.CRITICO,
            mensaje=mensaje,
            valor_actual=carga_futura_total,
            valor_limite=capacidad_minima,
            porcentaje_uso=porcentaje_uso,
            recomendacion=recomendacion
        )

    def check_capacidad_transformador(self, equipo: EquipoNuevo,
                                      datos: DatosNodoTiempoReal,
                                      impacto: Dict) -> ResultadoCheck:
        """
        CHECK: Verifica capacidad del transformador
        """
        kva_actual = datos.tr_kva_actual
        kva_futuro = kva_actual + impacto['kva_adicional']
        kva_max = self.limites['tr_kva_max']

        # Aplicar margen de seguridad
        kva_max_operativo = kva_max * (1 - self.config['margen_seguridad_tr'] / 100)

        aprobado = kva_futuro <= kva_max_operativo
        porcentaje_uso = (kva_futuro / kva_max) * 100

        mensaje = (
            f"Uso actual: {kva_actual:.1f} kVA. "
            f"Uso futuro: {kva_futuro:.1f} kVA. "
            f"Límite operativo (90%): {kva_max_operativo:.1f} kVA."
        )

        recomendacion = None
        if not aprobado:
            recomendacion = f"Instalar transformador de mayor capacidad o reducir carga en {(kva_futuro - kva_max_operativo):.1f} kVA."
        elif porcentaje_uso > 75:
            recomendacion = "Planificar upgrade de transformador en el corto plazo."

        return ResultadoCheck(
            nombre="Capacidad Transformador",
            aprobado=aprobado,
            severidad=NivelSeveridad.CRITICO if not aprobado else NivelSeveridad.ADVERTENCIA,
            mensaje=mensaje,
            valor_actual=kva_futuro,
            valor_limite=kva_max_operativo,
            porcentaje_uso=porcentaje_uso,
            recomendacion=recomendacion
        )

    def check_breaker_ml_rectificador(self, equipo: EquipoNuevo,
                                      datos: DatosNodoTiempoReal,
                                      impacto: Dict) -> ResultadoCheck:
        """
        CHECK: Verifica capacidad del breaker AC que alimenta los rectificadores

        MEJORA: Considera distribución de carga entre rectificadores
        """
        # Corriente AC actual estimada (total de ambos rectificadores)
        potencia_dc_actual = (datos.r1_amps_dc * datos.r1_voltaje_dc +
                              datos.r2_amps_dc * datos.r2_voltaje_dc)
        potencia_ac_actual = potencia_dc_actual / self.config['eficiencia_rectificador']

        corriente_ac_actual = potencia_ac_actual / (
            math.sqrt(3) * datos.ml_voltaje_ac_avg * datos.ml_fp
        )

        # Corriente AC futura
        corriente_ac_futura = corriente_ac_actual + impacto['corriente_ac_fase_a']

        # Límite del breaker con margen
        limite_breaker = self.limites['breaker_ml_por_rectificador_amps']
        limite_operativo = limite_breaker * (1 - self.config['margen_seguridad_breaker'] / 100)

        # NOTA: Aquí asumimos que hay un breaker por rectificador
        # La corriente se divide aproximadamente igual
        corriente_por_breaker = corriente_ac_futura / 2

        aprobado = corriente_por_breaker <= limite_operativo
        porcentaje_uso = (corriente_por_breaker / limite_breaker) * 100

        mensaje = (
            f"Corriente AC estimada por breaker: {corriente_por_breaker:.1f}A. "
            f"Límite operativo (80%): {limite_operativo:.1f}A. "
            f"Límite nominal: {limite_breaker}A."
        )

        recomendacion = None
        if not aprobado:
            recomendacion = "Reemplazar breakers por unos de mayor capacidad o redistribuir cargas."
        elif porcentaje_uso > 70:
            recomendacion = "Monitorear de cerca. Considerar upgrade de breakers."

        return ResultadoCheck(
            nombre="Breaker ML (por Rectificador)",
            aprobado=aprobado,
            severidad=NivelSeveridad.CRITICO,
            mensaje=mensaje,
            valor_actual=corriente_por_breaker,
            valor_limite=limite_operativo,
            porcentaje_uso=porcentaje_uso,
            recomendacion=recomendacion
        )

    def check_distribucion_carga_rectificadores(self, equipo: EquipoNuevo,
                                                datos: DatosNodoTiempoReal,
                                                impacto: Dict) -> ResultadoCheck:
        """
        CHECK: Verifica que la distribución de carga entre rectificadores sea balanceada
        """
        # Carga actual de cada rectificador (%)
        pct_r1 = datos.r1_porcentaje_carga
        pct_r2 = datos.r2_porcentaje_carga

        # Diferencia de carga
        diferencia_actual = abs(pct_r1 - pct_r2)

        # Límite recomendado: diferencia < 20%
        aprobado = diferencia_actual < 20

        mensaje = (
            f"Carga R1: {pct_r1:.1f}%, Carga R2: {pct_r2:.1f}%. "
            f"Diferencia: {diferencia_actual:.1f}%."
        )

        recomendacion = None
        if not aprobado:
            if pct_r1 > pct_r2:
                recomendacion = "Conectar el nuevo equipo preferentemente al PDB2 (alimentado por R2) para balancear."
            else:
                recomendacion = "Conectar el nuevo equipo preferentemente al PDB1 (alimentado por R1) para balancear."

        return ResultadoCheck(
            nombre="Balance de Carga entre Rectificadores",
            aprobado=aprobado,
            severidad=NivelSeveridad.ADVERTENCIA,
            mensaje=mensaje,
            valor_actual=diferencia_actual,
            valor_limite=20.0,
            porcentaje_uso=(diferencia_actual / 20.0) * 100,
            recomendacion=recomendacion
        )

    def check_voltaje_sistema(self, datos: DatosNodoTiempoReal) -> ResultadoCheck:
        """
        CHECK: Verifica que el voltaje DC esté dentro de rangos normales
        """
        # Voltaje promedio de ambos rectificadores
        voltaje_avg = (datos.r1_voltaje_dc + datos.r2_voltaje_dc) / 2

        v_min = self.config['voltaje_dc_min']
        v_max = self.config['voltaje_dc_max']
        v_nominal = self.config['voltaje_dc_nominal']

        aprobado = v_min <= voltaje_avg <= v_max

        # Desviación del nominal
        desviacion = abs(voltaje_avg - v_nominal) / v_nominal * 100

        mensaje = (
            f"Voltaje DC promedio: {voltaje_avg:.2f}V. "
            f"Rango permitido: {v_min}V - {v_max}V. "
            f"Desviación del nominal ({v_nominal}V): {desviacion:.1f}%."
        )

        recomendacion = None
        if not aprobado:
            if voltaje_avg < v_min:
                recomendacion = "Voltaje bajo. Revisar rectificadores y baterías."
            else:
                recomendacion = "Voltaje alto. Ajustar configuración de rectificadores."
        elif desviacion > 5:
            recomendacion = "Voltaje dentro de rango pero con desviación significativa. Monitorear."

        return ResultadoCheck(
            nombre="Voltaje DC Sistema",
            aprobado=aprobado,
            severidad=NivelSeveridad.ADVERTENCIA,
            mensaje=mensaje,
            valor_actual=voltaje_avg,
            valor_limite=v_max if voltaje_avg > v_nominal else v_min,
            porcentaje_uso=desviacion,
            recomendacion=recomendacion
        )

    def evaluar_instalacion(self, equipo: EquipoNuevo,
                           datos: DatosNodoTiempoReal,
                           modo_estricto: bool = True) -> Dict:
        """
        Evalúa si es viable instalar un nuevo equipo

        Args:
            equipo: Especificaciones del equipo a instalar
            datos: Datos en tiempo real del nodo
            modo_estricto: Si True, cualquier check CRÍTICO reprobado aborta

        Returns:
            Diccionario con resultado completo de la evaluación
        """
        # Calcular impacto
        impacto = self.calcular_impacto_equipo(equipo, datos)

        # Ejecutar todos los checks
        checks = [
            self.check_voltaje_sistema(datos),
            self.check_redundancia_n_mas_1(equipo, datos, impacto),
            self.check_capacidad_transformador(equipo, datos, impacto),
            self.check_breaker_ml_rectificador(equipo, datos, impacto),
            self.check_distribucion_carga_rectificadores(equipo, datos, impacto),
        ]

        # Evaluar resultado
        checks_criticos_fallidos = [c for c in checks if not c.aprobado and c.severidad == NivelSeveridad.CRITICO]
        checks_advertencias = [c for c in checks if not c.aprobado and c.severidad == NivelSeveridad.ADVERTENCIA]

        aprobado_final = len(checks_criticos_fallidos) == 0

        if modo_estricto and not aprobado_final:
            mensaje_final = "❌ INSTALACIÓN RECHAZADA - Existen checks críticos fallidos."
        elif aprobado_final and len(checks_advertencias) == 0:
            mensaje_final = "✅ INSTALACIÓN APROBADA - Todos los checks pasaron."
        elif aprobado_final and len(checks_advertencias) > 0:
            mensaje_final = f"⚠️ INSTALACIÓN APROBADA CON ADVERTENCIAS - {len(checks_advertencias)} advertencia(s)."
        else:
            mensaje_final = "❌ INSTALACIÓN RECHAZADA."

        return {
            "equipo": {
                "nombre": equipo.nombre,
                "potencia_w": equipo.potencia_watts,
                "pdb_destino": equipo.pdb_destino or "No especificado"
            },
            "aprobado": aprobado_final,
            "mensaje_final": mensaje_final,
            "impacto_calculado": impacto,
            "checks": checks,
            "resumen": {
                "total_checks": len(checks),
                "criticos_fallidos": len(checks_criticos_fallidos),
                "advertencias": len(checks_advertencias),
                "aprobados": len([c for c in checks if c.aprobado])
            }
        }

    def generar_reporte_html(self, resultado: Dict) -> str:
        """Genera un reporte HTML del resultado (para futuro uso)"""
        # Implementación futura
        pass

    def exportar_a_json(self, resultado: Dict, archivo: str):
        """Exporta el resultado a un archivo JSON"""
        # Convertir ResultadoCheck a dict
        resultado_serializable = {
            **resultado,
            "checks": [
                {
                    "nombre": c.nombre,
                    "aprobado": c.aprobado,
                    "severidad": c.severidad.value,
                    "mensaje": c.mensaje,
                    "valor_actual": c.valor_actual,
                    "valor_limite": c.valor_limite,
                    "porcentaje_uso": c.porcentaje_uso,
                    "recomendacion": c.recomendacion
                }
                for c in resultado["checks"]
            ]
        }

        with open(archivo, 'w', encoding='utf-8') as f:
            json.dump(resultado_serializable, f, indent=2, ensure_ascii=False)


# ==========================================
# EJEMPLO DE USO CON TUS DATOS REALES
# ==========================================

if __name__ == "__main__":
    # Crear evaluador
    evaluador = EvaluadorNodoIdeo()

    # Datos del nodo en tiempo real (tus datos)
    datos_nodo = DatosNodoTiempoReal(
        # Transformador
        tr_kva_actual=21.0,

        # Main Line
        ml_amps_ac_r=58.0,
        ml_amps_ac_s=58.0,
        ml_amps_ac_t=59.0,
        ml_voltaje_ac_rs=214.0,
        ml_voltaje_ac_st=214.0,
        ml_voltaje_ac_tr=215.0,
        ml_fp=0.98,

        # Rectificador 1
        r1_amps_dc=119.0,
        r1_voltaje_dc=54.47,
        r1_modulos_instalados=21,
        r1_modulos_fallados=0,
        r1_porcentaje_carga=10,  # Estimado

        # Rectificador 2
        r2_amps_dc=157.0,
        r2_voltaje_dc=54.47,
        r2_modulos_instalados=19,
        r2_modulos_fallados=0,
        r2_porcentaje_carga=14,  # Estimado
    )

    # Equipo a instalar
    equipo_nuevo = EquipoNuevo(
        nombre="Switch Cisco Catalyst 9300",
        potencia_watts=200,  # Típico de un switch de 48 puertos
        voltaje_operacion=54,
        redundancia_requerida=True,
        pdb_destino="PDB2"  # Sugerencia para balancear
    )

    # Evaluar instalación
    print("=" * 80)
    print(f"EVALUACIÓN DE INSTALACIÓN - NODO IDEO CALI")
    print(f"Equipo: {equipo_nuevo.nombre} ({equipo_nuevo.potencia_watts}W)")
    print("=" * 80)

    resultado = evaluador.evaluar_instalacion(equipo_nuevo, datos_nodo, modo_estricto=True)

    print(f"\n{resultado['mensaje_final']}")
    print(f"\nResumen: {resultado['resumen']['aprobados']}/{resultado['resumen']['total_checks']} checks aprobados")

    print("\n" + "-" * 80)
    print("IMPACTO CALCULADO:")
    print("-" * 80)
    for key, value in resultado['impacto_calculado'].items():
        print(f"  {key}: {value:.2f}")

    print("\n" + "-" * 80)
    print("RESULTADOS DE CHECKS:")
    print("-" * 80)
    for check in resultado['checks']:
        icono = "✅" if check.aprobado else "❌"
        severidad = f"[{check.severidad.value}]"

        print(f"\n{icono} {check.nombre} {severidad}")
        print(f"   {check.mensaje}")
        print(f"   Uso: {check.porcentaje_uso:.1f}% ({check.valor_actual:.2f} / {check.valor_limite:.2f})")

        if check.recomendacion:
            print(f"   💡 Recomendación: {check.recomendacion}")

    print("\n" + "=" * 80)
    print("FIN DE EVALUACIÓN")
    print("=" * 80)

    # Exportar resultado a JSON
    evaluador.exportar_a_json(resultado, 'evaluacion_resultado.json')
    print("\n📄 Resultado exportado a: evaluacion_resultado.json")

EVALUACIÓN DE INSTALACIÓN - NODO IDEO CALI
Equipo: Switch Cisco Catalyst 9300 (200W)

✅ INSTALACIÓN APROBADA - Todos los checks pasaron.

Resumen: 5/5 checks aprobados

--------------------------------------------------------------------------------
IMPACTO CALCULADO:
--------------------------------------------------------------------------------
  corriente_dc_a: 3.70
  potencia_dc_kw: 0.20
  potencia_ac_w: 212.77
  potencia_ac_kw: 0.21
  corriente_ac_fase_a: 0.58
  kva_adicional: 0.22

--------------------------------------------------------------------------------
RESULTADOS DE CHECKS:
--------------------------------------------------------------------------------

✅ Voltaje DC Sistema [ADVERTENCIA]
   Voltaje DC promedio: 54.47V. Rango permitido: 43.0V - 57.0V. Desviación del nominal (48.0V): 13.5%.
   Uso: 13.5% (54.47 / 57.00)
   💡 Recomendación: Voltaje dentro de rango pero con desviación significativa. Monitorear.

✅ Redundancia N+1 (Rectificadores) [CRÍTICO]
   Carga futura 